In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import datetime
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import re
import numpy as np
from sympy.abc import x
from networkx.algorithms.community import greedy_modularity_communities, modularity
from tqdm import tqdm
from collections import deque
import csv 
from community import best_partition
import random
import datetime
from collections import defaultdict

# 常量配置
CTWALKS_PARAMS = {
    'num_walks': 10,        # 每个时间窗口的随机游走次数
    'walk_length': 5,        # 每次游走的步长
    'sampling_rate': 0.3,    # 节点采样率
    'max_attempts': 3       # 无效节点重试次数
}

In [2]:
G = nx.DiGraph()
table = pd.read_csv("final_without_quotes.txt", encoding='utf-8')
table = table.iloc[:, 1:]  # 移除第一列（索引列）

In [3]:
to_drop = []  # 用于存储需要删除的行索引
for i in range(len(table)):
    try:
        table.loc[i, 'raw_value.created_at'] = pd.to_datetime(table.loc[i, 'raw_value.created_at'])
    except:
        to_drop.append(i)
    if isinstance(table.loc[i, 'raw_value.created_at'], datetime.datetime):
        pass
    else:
        to_drop.append(i)

    if table.loc[i, 'raw_value.created_at'].tzinfo is not None:
        to_drop.append(i)

In [4]:
table.drop(to_drop, inplace=True)

table['raw_value.created_at'].sort_values(ascending=True)
table.to_csv("final_without_quotes.txt", encoding='utf-8')
table.index = table['raw_value.created_at']


/home/root_gdut/miniconda3/envs/network/lib/python3.9/site-packages/pandas/core/indexes/base.py:7631: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(index_like, name=name, copy=copy)


In [5]:
# 定义数据存储结构
users = {i: {} for i in set(table['raw_value.user_id_str'].values.tolist())}
makelist = {}
date_diff = 30  # 时间差阈值（30天）
output_dir = "degree"
edge_timestamps = deque()  # 双端队列缓存边的时间戳和对应的边
edge_lifetime = 30  # 边的生命周期 (天)

# 用于批量存储index.csv中的数据
index_buffer = []  # 用于存储一段时间内的写入数据

with open('network_metrics.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['date', 'nodes', 'edges', 'modularity', 
                 'assortativity', 'intra_ratio', 'inter_ratio']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for w, w_t in table.resample('D'):  # 按天分组
        if len(w_t) > 0:  # 如果本天有数据
            for index, row in w_t.iterrows():
                tweet_id = row['raw_value.id_str']  # 获取推文ID
                # 判断推文是否存在交互关系
                if row['raw_value.in_reply_to_status_id_str'] or row['raw_value.quoted_status_id_str']:
                    this_data_datetime = row['raw_value.created_at']
                    
                    # 检查用户ID是否为空
                    user_id = row['raw_value.user_id_str']
                    if pd.isna(user_id) or user_id == "":
                        continue  # 跳过没有有效用户ID的记录

                    # 添加节点到图中
                    G.add_node(tweet_id)  # 添加节点到图中

                    # 遍历缓存中的推文，寻找交互关系
                    for make_i in makelist.keys():
                        diff = (this_data_datetime - makelist[make_i]['created_at']).days
                        if diff < date_diff:  # 如果时间差在范围内
                            # 回复关系
                            if is_valid_id(row['raw_value.in_reply_to_status_id_str']):
                                edge_timestamps.append((makelist[make_i]['id'], row['raw_value.in_reply_to_status_id_str'], this_data_datetime))
                                G.add_edge(makelist[make_i]['id'], row['raw_value.in_reply_to_status_id_str'], weight=1, timestamp=this_data_datetime)

                            if is_valid_id(row['raw_value.quoted_status_id_str']):
                                edge_timestamps.append((makelist[make_i]['id'], row['raw_value.quoted_status_id_str'], this_data_datetime))
                                G.add_edge(makelist[make_i]['id'], row['raw_value.quoted_status_id_str'], weight=1, timestamp=this_data_datetime)

            if row['raw_value.user_id_str'] not in makelist:
                makelist[row['raw_value.user_id_str']] = {
                    'userid': row['raw_value.user_id_str'],
                    'id': tweet_id,
                    'text': row['raw_value.full_text'],
                    'reply': row['raw_value.in_reply_to_status_id_str'],
                    'quoted': row['raw_value.quoted_status_id_str'],
                    'cleantext': row['raw_value.full_text'],
                    'created_at': row['raw_value.created_at']
                }

            # 删除超过30天的边和节点
            to_remove_edges = []
            to_remove_nodes = set()  # 用于存储需要删除的节点

            # 从双端队列中删除过期的边
            while edge_timestamps and (w - edge_timestamps[0][2]).days > edge_lifetime:
                edge_start, edge_end, timestamp = edge_timestamps.popleft()
                if G.has_edge(edge_start, edge_end):
                    G.remove_edge(edge_start, edge_end)
                    to_remove_nodes.add(edge_start)
                    to_remove_nodes.add(edge_end)
                    print(f"已删除边：{(edge_start, edge_end)}  (时间差超过 {date_diff} 天)")

            # 删除节点
            for node in tqdm(to_remove_nodes, desc="删除过期节点", unit="节点"):
                if G.has_node(node):
                    G.remove_node(node)
                    print(f"已删除节点：{node}  (时间差超过 {date_diff} 天)")

# 计算网络指标
            metrics = {
                'nodes': G.number_of_nodes(),
                'edges': G.number_of_edges()
            }
            
            if metrics['edges'] > 0:
                try:
                    # 计算模块度
                    undirected_G = G.to_undirected()
                    partition = best_partition(undirected_G)
                    community_dict = defaultdict(set)
                    for node, comm_id in partition.items():
                        community_dict[comm_id].add(node)
                    communities = list(community_dict.values())
                    metrics['modularity'] = modularity(undirected_G, communities)
                    
                    # 计算同配性
                    metrics['assortativity'] = nx.degree_assortativity_coefficient(G)
                    
                    # 计算CTWalks指标
                    intra, inter = compute_ctwalks_metrics(G)
                    metrics['intra_ratio'] = intra
                    metrics['inter_ratio'] = inter
                except Exception as e:
                    print(f"指标计算错误: {str(e)}")
                    metrics.update({'modularity': 0, 'assortativity': 0,
                                'intra_ratio': 0, 'inter_ratio': 0})
            else:
                metrics.update({'modularity': 0, 'assortativity': 0,
                            'intra_ratio': 0, 'inter_ratio': 0})
            # 将指标写入CSV文件
            writer.writerow(metrics)

# 最终清理
G_clean = validate_gexf_attributes(G)
nx.write_gexf(G, "final_network.gexf")
print("处理完成。结果保存在network_metrics.csv和network_viz/目录")

删除过期节点: 0节点 [00:00, ?节点/s]


NameError: name 'is_valid_id' is not defined

In [ ]:
# 隐式
import pandas as pd
import os
import json
import networkx as nx
import datetime
import torch
from networkx.algorithms.community import greedy_modularity_communities, modularity
import sympy
from collections import deque
import numpy as np
from tqdm import tqdm
import scipy.integrate as integrate
import csv 
def load_vectors(file_path):
    vector_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(',', 1)  # 按逗号分割，第一部分是推文ID，第二部分是向量
            tweet_id = parts[0]  # 获取推文ID
            try:
                # 将向量部分按空格拆分并转换为浮动数值
                vector = np.array([float(x) for x in parts[1].split()])  # 向量部分按空格拆分
                vector_dict[tweet_id] = vector
            except ValueError as e:
                print(f"无法将以下值转换为浮动数值: {parts[1]}. 错误信息: {e}")
    return vector_dict

# 调用方法
vector_dict = load_vectors("/home/root_gdut/KDD2025/old/embeddin/output_vectors.txt")
print(f"已加载的推文ID数量: {len(vector_dict)}")
print(f"部分推文ID: {list(vector_dict.keys())[:10]}")  # 打印前10个ID

In [7]:
import pandas as pd
import os
os.chdir(r'/home/root_gdut/KDD2025/old/embeddin')
import json
import networkx as nx
import datetime
import torch
from networkx.algorithms.community import greedy_modularity_communities, modularity
import sympy
from collections import deque
import numpy as np
from tqdm import tqdm
import scipy.integrate as integrate
import csv 
def load_vectors(file_path):
    vector_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(',', 1)  # 按逗号分割，第一部分是推文ID，第二部分是向量
            tweet_id = parts[0]  # 获取推文ID
            try:
                # 将向量部分按空格拆分并转换为浮动数值
                vector = np.array([float(x) for x in parts[1].split()])  # 向量部分按空格拆分
                vector_dict[tweet_id] = vector
            except ValueError as e:
                print(f"无法将以下值转换为浮动数值: {parts[1]}. 错误信息: {e}")
    return vector_dict

# 调用方法
vector_dict = load_vectors("output_vectors.txt")
print(f"已加载的推文ID数量: {len(vector_dict)}")
print(f"部分推文ID: {list(vector_dict.keys())[:10]}")  # 打印前10个ID


scores = 0.90
sentiment_df = pd.read_csv('final_with_sentiment.csv')

# 创建一个字典，方便通过 `id_str` 查找情感概率
sentiment_dict = {
    row['raw_value.id_str']: {
        'negative_probability': row['negative_probability'],
        'neutral_probability': row['neutral_probability'],
        'positive_probability': row['positive_probability']
    }
    for _, row in sentiment_df.iterrows()
}




def calculate_daw(time_diff_days):
    """根据时间差计算动态衰减指数DAW"""
    # 计算衰减因子，时间差转化为月单位
    time_diff_months = abs(time_diff_days) / 30

    # 使用数值积分替代符号积分
    result, error = integrate.quad(lambda x: np.exp(-x), 0, time_diff_months)
    
    # 计算DAW值，1减去积分结果
    daw_value = 1 - result
    
    # 保证DAW的下限为0.01
    return daw_value if daw_value >= 0.01 else 0.01



# 更新vol函数，整合DAW计算
def vol(parent_raws, emtions1, retweet_count, quote_count, favorite_count, reply_count,
        max_retweet_count, max_quote_count, max_favorite_count, max_reply_count, current_time):
    emtions = emtions1
    vol1 = 0
    if parent_raws:
        for raw in parent_raws:  # 0:相似度，1：相差时间，2：情感得分
            # 确保 raw[1] 是 pandas Timestamp 类型
            raw_time = pd.to_datetime(raw[1])  # 将父节点时间转换为 Timestamp 类型
            current_time = pd.to_datetime(current_time)  # 确保 current_time 是 Timestamp 类型
            
            # 计算时间差（天数）
            time_diff = (current_time - raw_time).days  # 获取天数差
            daw_value = calculate_daw(time_diff)  # 计算DAW
            vol1 += daw_value * raw[0] * raw[2]  # 根据DAW、相似度、情感得分计算传播强度
        vol1 /= len(parent_raws)
    # 父节点的平均情感 + 子节点情感 * 子节点的相关指标
    print('emotion:', emtions, 'volumn:', vol1)
    emtions += vol1
    if emtions < -1:
        emtions = -1
    elif emtions > 1:
        emtions = 1
    return emtions

def emtions(tweet_id):
    # 直接通过 tweet_id 获取情感概率
    sentiment_probs = sentiment_dict.get(tweet_id)
    if sentiment_probs:
        negative_prob = sentiment_probs['negative_probability']
        neutral_prob = sentiment_probs['neutral_probability']
        positive_prob = sentiment_probs['positive_probability']

        # 计算情感分数
        if positive_prob > neutral_prob and positive_prob > negative_prob:
            sentiment_score = positive_prob
        elif negative_prob > neutral_prob and negative_prob > positive_prob:
            sentiment_score = -negative_prob
        elif neutral_prob > positive_prob and neutral_prob > negative_prob:
            if positive_prob > negative_prob:
                sentiment_score = 1 - neutral_prob
            else:
                sentiment_score = -(1 - neutral_prob)
    else:
        sentiment_score = 0  # 如果找不到情感概率，返回默认值（例如0）

    return sentiment_score

def bert_va(tweet_id, emtions1):
    tweet_id = str(tweet_id)  # 确保tweet_id是字符串类型
    if tweet_id not in vector_dict:
        raise ValueError(f"向量文件中找不到推文ID {tweet_id}")
    
    tweet_vector = vector_dict[tweet_id]
    embedding_with_emotion = np.concatenate((tweet_vector, np.array([emtions1])), axis=0)
    
    return embedding_with_emotion


def cosine(x, y):
    # 确保x和y是tensor类型
    x = torch.tensor(x)
    y = torch.tensor(y)
    
    # 计算余弦相似度
    return torch.nn.functional.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0))

# 新增社区检测函数
def detect_communities(G):
    """处理有向图的社区检测"""
    from community import best_partition
    undirected_G = nx.Graph()
    # 聚合有向边到无向图
    for u, v, data in G.edges(data=True):
        if undirected_G.has_edge(u, v):
            undirected_G[u][v]['weight'] += data.get('weight', 1.0)
        else:
            undirected_G.add_edge(u, v, weight=data.get('weight', 1.0))
    partition = best_partition(undirected_G, weight='weight')
    nx.set_node_attributes(G, partition, 'community')
    return partition

# Assuming these functions are already defined elsewhere in your code
# emtions(), bert_va(), cosine(), calculate_daw(), vol(), etc.

import csv
from collections import deque

def main2():
    G = nx.DiGraph()

    # 读取数据并进行初步处理
    table = pd.read_csv("final.txt")
    table = table.iloc[:, 1:]  # 删除第一列，如果是索引列
    table['raw_value.created_at'] = pd.to_datetime(table['raw_value.created_at'])

    # 确保数据按创建时间排序
    table = table.sort_values(by='raw_value.created_at')

    table.index = table['raw_value.created_at']  # 将时间列设置为索引
    # 初始化数据结构
    users = {i: {} for i in set(table['raw_value.user_id_str'].values.tolist())}
    makelist = {}
    emotionlist = {}
    date_diff = 30  # 删除节点的时间差上限为30天

    # 获取最大转发、引用、点赞和评论数量
    max_retweet_count, max_quote_count, max_favorite_count, max_reply_count = (
        table['raw_value.retweet_count'].max(),
        table['raw_value.quote_count'].max(),
        table['raw_value.favorite_count'].max(),
        table['raw_value.reply_count'].max()
    )
    tweet_to_user = {}
    # 使用双端队列缓存边的时间戳和对应的边
    edge_timestamps = deque()  # 用于存储边的时间戳和对应的边
    # 用于缓存时间段的计算结果
    data_buffer = []  # 存储每个时间点的指标数据
    # 打开 edge_log.csv 文件，用于记录连边信息
    with open("edge_log.csv", "a+", newline='', encoding="utf-8") as edge_file:
        edge_writer = csv.writer(edge_file)
        # 如果文件为空，写入表头
        if edge_file.tell() == 0:
            edge_writer.writerow(['Source_Node', 'Target_Node', 'Timestamp'])
        # 按天分组数据进行处理
        for w, w_t in table.resample('D'):  # 每天处理一次数据
            if len(w_t) == 0:
                continue  # 如果这一天没有数据，跳过该天的处理

            current_time = w  # 使用当前时间作为当前时间
            new_edges_added = False  # 标记这一天是否有新增边或节点

            # 处理每一条推文数据
            for index, row in w_t.iterrows():
                strs = row['raw_value.full_text']
                strs2 = row['Tags']
                tweet_id = row['raw_value.id_str']  # 获取 tweet_id

                # 获取情感分数
                emtions1 = emtions(tweet_id)
                bert_va1 = bert_va(tweet_id, emtions1)
                this_data_datetime = row['raw_value.created_at']

                G.add_node(row['raw_value.id_str'])
                tweet_to_user[row['raw_value.id_str']] = row['raw_value.user_id_str']
                parent = []  # 存储相关的父节点

                # 遍历当前的 makelist，检查已有的边是否超过30天
                for make_i in makelist.keys():
                    diff = (this_data_datetime - makelist[make_i]['created_at']).days
                    if diff < date_diff:  # 如果时间差小于30天，继续检查
                        daw_value = calculate_daw(diff)
                        score = float(cosine(bert_va1, makelist[make_i]['pooled']) * daw_value)

                        if 1 > score > scores:  # 相似度阈值
                            # 添加带有时间戳的边
                            G.add_edge(makelist[make_i]['id'], row['raw_value.id_str'], weight=round(score, 2), timestamp=this_data_datetime)
                            
                            # 记录边的ID和时间戳到文件
                            edge_writer.writerow([makelist[make_i]['id'], row['raw_value.id_str'], this_data_datetime])
                            new_edges_added = True

                # 更新情感值
                emtions1 = vol(parent, emtions1, row['raw_value.retweet_count'], row['raw_value.quote_count'],
                               row['raw_value.favorite_count'], row['raw_value.reply_count'],
                               max_retweet_count, max_quote_count, max_favorite_count, max_reply_count, current_time)

                if w not in users[row['raw_value.user_id_str']].keys():
                    users[row['raw_value.user_id_str']][w] = [emtions1]
                else:
                    users[row['raw_value.user_id_str']][w].append(emtions1)

                # 更新节点信息
                makelist[row['raw_value.id_str']] = {
                    'userid': row['raw_value.user_id_str'],
                    'id': row['raw_value.id_str'],
                    'text': row['raw_value.full_text'],
                    'cleantext': strs,
                    'emotion': emtions1,
                    'pooled': bert_va1,
                    'created_at': row['raw_value.created_at']
                }
            # 即使没有新节点或边，也要删除超过30天的节点和它们的边
            to_remove_nodes = set()  # 用于存储需要删除的节点
            # 检查每个节点是否存在超过30天
            for node in list(G.nodes):  # 使用 list() 避免在遍历中修改节点集合
                node_creation_time = makelist[node]['created_at']
                node_age = (current_time - node_creation_time).days
                if node_age > date_diff:  # 如果节点存在超过30天
                    to_remove_nodes.add(node)
            # 删除超过30天的节点及其所有边
            for node in tqdm(to_remove_nodes, desc="删除过期节点", unit="节点"):
                if G.has_node(node):
                    # 删除与该节点相关的所有边
                    G.remove_node(node)
                    print(f"已删除节点及其所有边：{node}  (时间差超过 {date_diff} 天)")
            # 删除节点和边后，及时更新数据结构
            for node in to_remove_nodes:
                if node in makelist:
                    del makelist[node]
                if node in tweet_to_user:
                    del tweet_to_user[node]
            # 计算网络指标并缓存到内存
            if G.number_of_edges() > 0:
                d = dict(nx.degree(G))
                d_avg = sum(d.values()) / len(G.nodes)
                x = list(range(max(d.values()) + 1))
                d_list = nx.degree_histogram(G)
                y = np.array(d_list) / len(G)
                # 检测社区并计算模块化程度
                communities = greedy_modularity_communities(G.to_undirected())
                modularity_score = modularity(G.to_undirected(), communities, weight=None)
                # 在每天处理前执行社区检测（示例每天检测，实际可按需调整频率）
                for w, w_t in table.resample('D'):
                    if len(w_t) == 0:
                        continue
                    
                    # 当天新增边后的社区检测
                    detect_communities(G)
                    
                    # 计算社区指标
                    communities = {}  # 格式：{社区ID: 节点列表}
                    for node in G.nodes():
                        comm = G.nodes[node].get('community', -1)
                        communities.setdefault(comm, []).append(node)
                    
                    # 统计社区内外交互
                    intra_edges = 0
                    inter_edges = 0
                    node_communities = {}
                    bridging_nodes = set()
                    
                    for u, v, data in G.edges(data=True):
                        u_comm = G.nodes[u].get('community', -1)
                        v_comm = G.nodes[v].get('community', -1)
                        # 统计节点连接的社区
                        node_communities.setdefault(u, set()).add(v_comm)
                        node_communities.setdefault(v, set()).add(u_comm)
                        if u_comm == v_comm:
                            intra_edges += data.get('weight', 1)
                        else:
                            inter_edges += data.get('weight', 1)
                    # 精确桥接节点：连接超过1个社区的节点
                    bridging_nodes = {
                        node for node, comms in node_communities.items()
                        if len(comms) > 1
                    }
                q1 = G.number_of_nodes()  # 节点数
                q2 = G.number_of_edges()  # 边数
                q3 = nx.pagerank(G,alpha=0.85)
                q6 = nx.degree_assortativity_coefficient(G)  # 度同配系数
                # 将计算结果缓存到 data_buffer
                data_buffer.append([w.strftime('%Y-%m-%d'), q1, q2, q3, q6, modularity_score,intra_edges,inter_edges,len(bridging_nodes),len(communities)])
            # 批量写入文件
            if len(data_buffer) >= 10:  # 如果缓存的记录达到10个（可以根据需要调整）
                with open("index.csv", 'a+', newline='', encoding='utf-8') as f:
                    writer = csv.writer(f)
                    if f.tell() == 0:  # 如果文件为空，先写入表头
                        writer.writerow(['Time', 'Num_nodes', 'Num_edges', 'PageRank', 'Degree_assortativity', 'Modularity',
                                         'Intra_edges','Inter_edges','Bridging_nodes','Community_count'])
                    writer.writerows(data_buffer)  # 批量写入
                data_buffer.clear()  # 清空缓存

                # 可选：输出日志
                print(f"已批量写入 {len(data_buffer)} 条数据到 index.csv")

            timestamp_str = w.strftime("%Y-%m-%d")
            adjacency_matrix = nx.adjacency_matrix(G).todense()
            pd.DataFrame(adjacency_matrix, index=G.nodes, columns=G.nodes).to_csv(f'G-{timestamp_str}.csv')

            tweet_network_filename = f'tweet_network-{timestamp_str}.edgelist'
            nx.write_edgelist(G, tweet_network_filename)

            with open("final_emotions.csv", 'a+') as f:
                f.write(f"{timestamp_str},{emtions1}\n")
    # 保存映射关系和其他信息
    with open("emotionall.csv", "w+", encoding='utf-8') as f:
        f.write(str(makelist))
    # 保存tweet_to_user映射
    with open("tweet_to_user_mapping.json", "w") as f:
        json.dump(tweet_to_user, f)
    return users

# 主函数执行
if __name__ == '__main__':
    print(datetime.datetime.now())
    main2()


已加载的推文ID数量: 13250
部分推文ID: ['1371366321309556736', '1373893479160549376', '1377151142170222592', '1382037657291735040', '1384423637067911168', '1384656967998656512', '1386254705878310912', '1387712331891757056', '1388692619727884288', '1390579956292411392']
2025-02-04 04:10:06.267576
emotion: -0.3555069863796234 volumn: 0


删除过期节点: 0节点 [00:00, ?节点/s]


emotion: 0.3658796548843384 volumn: 0


删除过期节点: 0节点 [00:00, ?节点/s]


emotion: -0.6165007948875427 volumn: 0


...

已删除节点及其所有边：1453149734470889472  (时间差超过 30 天)
已删除节点及其所有边：1453152673323274240  (时间差超过 30 天)
已删除节点及其所有边：1453157537138814976  (时间差超过 30 天)
已删除节点及其所有边：1453169119549800448  (时间差超过 30 天)
已删除节点及其所有边：1453168491947720704  (时间差超过 30 天)
已删除节点及其所有边：1453169192828424192  (时间差超过 30 天)
已删除节点及其所有边：1453170552475033600  (时间差超过 30 天)
已删除节点及其所有边：1453174016856903680  (时间差超过 30 天)
已删除节点及其所有边：1453199503423524864  (时间差超过 30 天)
已删除节点及其所有边：1453249144164921088  (时间差超过 30 天)
已删除节点及其所有边：1453254577248251904  (时间差超过 30 天)
已删除节点及其所有边：1453284357297291264  (时间差超过 30 天)
已删除节点及其所有边：1453327152057831424  (时间差超过 30 天)
已删除节点及其所有边：1453316461082992640  (时间差超过 30 天)
已删除节点及其所有边：1453349138653798400  (时间差超过 30 天)
已删除节点及其所有边：1453414238022561792  (时间差超过 30 天)
已删除节点及其所有边：1453348796478464000  (时间差超过 30 天)
已删除节点及其所有边：1453284359419670528  (时间差超过 30 天)
已删除节点及其所有边：1453295433284005888  (时间差超过 30 天)
已删除节点及其所有边：1453394414647197696  (时间差超过 30 天)
已删除节点及其所有边：1453349052196683776  (时间差超过 30 天)
已删除节点及其所有边：1453256541566640128  (时间差超过 30 天)
已删除节点及其所有边

emotion: -0.3759752810001373 volumn: 0
emotion: -0.346197634935379 volumn: 0
emotion: -0.5976843535900116 volumn: 0
emotion: -0.3489759266376495 volumn: 0
emotion: -0.4488156735897064 volumn: 0
emotion: -0.5965235531330109 volumn: 0
emotion: 0.3882107138633728 volumn: 0
emotion: -0.3443359136581421 volumn: 0
emotion: 0.3495221138000488 volumn: 0
emotion: 0.3530071675777435 volumn: 0
emotion: -0.4015910923480987 volumn: 0
emotion: 0.6507547497749329 volumn: 0
emotion: -0.4356249272823334 volumn: 0
emotion: -0.4782648682594299 volumn: 0
emotion: 0.3709233105182647 volumn: 0
emotion: 0.6380236148834229 volumn: 0
emotion: 0.3700376152992248 volumn: 0
emotion: -0.3583605289459228 volumn: 0
emotion: -0.3784521818161011 volumn: 0
emotion: -0.3861847817897796 volumn: 0
emotion: -0.3574851155281067 volumn: 0
emotion: 0.346022367477417 volumn: 0
emotion: 0.3939952850341797 volumn: 0
emotion: -0.6594472229480743 volumn: 0
emotion: 0.3988392353057861 volumn: 0
emotion: 0.648419976234436 volumn: 0


删除过期节点: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 103289.81节点/s]

已删除节点及其所有边：1453543902775062528  (时间差超过 30 天)
已删除节点及其所有边：1453546265539530752  (时间差超过 30 天)
已删除节点及其所有边：1453546267422846976  (时间差超过 30 天)
已删除节点及其所有边：1453546269326987264  (时间差超过 30 天)
已删除节点及其所有边：1453546274024611840  (时间差超过 30 天)
已删除节点及其所有边：1453546271403229184  (时间差超过 30 天)
已删除节点及其所有边：1453546279535996928  (时间差超过 30 天)
已删除节点及其所有边：1453546285449940992  (时间差超过 30 天)
已删除节点及其所有边：1453559206997479424  (时间差超过 30 天)
已删除节点及其所有边：1453567686269566976  (时间差超过 30 天)
已删除节点及其所有边：1453596747838722048  (时间差超过 30 天)
已删除节点及其所有边：1453625546445709312  (时间差超过 30 天)
已删除节点及其所有边：1453625550052810752  (时间差超过 30 天)
已删除节点及其所有边：1453737713132134400  (时间差超过 30 天)
已删除节点及其所有边：1453625541630652416  (时间差超过 30 天)
已删除节点及其所有边：1453638639221501952  (时间差超过 30 天)
已删除节点及其所有边：1453625544235356160  (时间差超过 30 天)
已删除节点及其所有边：1453632085512626176  (时间差超过 30 天)
已删除节点及其所有边：1453857954457923584  (时间差超过 30 天)
已删除节点及其所有边：1453638817257189376  (时间差超过 30 天)
已删除节点及其所有边：1453641565050597376  (时间差超过 30 天)
已删除节点及其所有边：1453643146189959168  (时间差超过 30 天)
已删除节点及其所有边

OSError: [Errno 28] No space left on device

In [1]:
import os
os.chdir(r'/home/root_gdut/KDD2025/old/old')
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import datetime
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np
from sympy.abc import x
from networkx.algorithms.community import greedy_modularity_communities, modularity
from tqdm import tqdm
from collections import deque
import csv
import community  # 添加python-louvain包

def is_valid_id(tweet_id):
    if pd.isna(tweet_id) or tweet_id == "" or not isinstance(tweet_id, (int, float)):
        return False
    return True

def main2():
    G = nx.DiGraph()
    table = pd.read_csv("final_without_quotes.txt", encoding='utf-8')
    table = table.iloc[:, 1:]
    
    to_drop = []
    for i in range(len(table)):
        try:
            table.loc[i, 'raw_value.created_at'] = pd.to_datetime(table.loc[i, 'raw_value.created_at'])
        except:
            to_drop.append(i)
        if not isinstance(table.loc[i, 'raw_value.created_at'], datetime.datetime) or table.loc[i, 'raw_value.created_at'].tzinfo is not None:
            to_drop.append(i)
    table.drop(to_drop, inplace=True)
    table['raw_value.created_at'].sort_values(ascending=True)
    table.to_csv("final_without_quotes.txt", encoding='utf-8')
    table.index = table['raw_value.created_at']

    users = {i: {} for i in set(table['raw_value.user_id_str'].values.tolist())}
    makelist = {}
    date_diff = 30
    output_dir = "degree"
    edge_timestamps = deque()
    edge_lifetime = 30
    index_buffer = []

    for w, w_t in table.resample('D'):
        if len(w_t) > 0:
            for index, row in w_t.iterrows():
                tweet_id = row['raw_value.id_str']
                if row['raw_value.in_reply_to_status_id_str'] or row['raw_value.quoted_status_id_str']:
                    this_data_datetime = row['raw_value.created_at']
                    user_id = row['raw_value.user_id_str']
                    if pd.isna(user_id) or user_id == "":
                        continue
                    G.add_node(tweet_id)
                    for make_i in makelist.keys():
                        diff = (this_data_datetime - makelist[make_i]['created_at']).days
                        if diff < date_diff:
                            # 处理回复关系
                            if is_valid_id(row['raw_value.in_reply_to_status_id_str']):
                                u = makelist[make_i]['id']
                                v = row['raw_value.in_reply_to_status_id_str']
                                if G.has_edge(u, v):
                                    G[u][v]['weight'] += 1  # 权重累加
                                else:
                                    G.add_edge(u, v, weight=1, timestamp=this_data_datetime)
                                edge_timestamps.append((u, v, this_data_datetime))
                            
                            # 处理引用关系
                            if is_valid_id(row['raw_value.quoted_status_id_str']):
                                u = makelist[make_i]['id']
                                v = row['raw_value.quoted_status_id_str']
                                if G.has_edge(u, v):
                                    G[u][v]['weight'] += 1  # 权重累加
                                else:
                                    G.add_edge(u, v, weight=1, timestamp=this_data_datetime)
                                edge_timestamps.append((u, v, this_data_datetime))

                    if row['raw_value.user_id_str'] not in makelist:
                        makelist[row['raw_value.user_id_str']] = {
                            'userid': row['raw_value.user_id_str'],
                            'id': tweet_id,
                            'text': row['raw_value.full_text'],
                            'reply': row['raw_value.in_reply_to_status_id_str'],
                            'quoted': row['raw_value.quoted_status_id_str'],
                            'cleantext': row['raw_value.full_text'],
                            'created_at': row['raw_value.created_at']
                        }

            # 删除过期边和节点
            while edge_timestamps and (w - edge_timestamps[0][2]).days > edge_lifetime:
                edge_start, edge_end, timestamp = edge_timestamps.popleft()
                if G.has_edge(edge_start, edge_end):
                    G.remove_edge(edge_start, edge_end)
                    if G.in_degree(edge_start) == 0 and G.out_degree(edge_start) == 0:
                        G.remove_node(edge_start)
                    if G.in_degree(edge_end) == 0 and G.out_degree(edge_end) == 0:
                        G.remove_node(edge_end)

            # 计算网络指标
            if G.number_of_nodes() > 0:
                # 度分布绘图代码保持不变...
                
                # 社区检测和统计
                intra_weight = 0
                inter_weight = 0
                num_communities = 0
                if G.number_of_edges() > 0:
                    # 构建无向加权图
                    undirected_G = nx.Graph()
                    for u, v, data in G.edges(data=True):
                        we = data.get('weight', 1)
                        if undirected_G.has_edge(u, v):
                            undirected_G[u][v]['weight'] += we
                        else:
                            undirected_G.add_edge(u, v, weight=we)
                    
                    # Louvain社区检测
                    try:
                        partition = community.best_partition(undirected_G, weight='weight')
                        communities_dict = {}
                        for node, comm_id in partition.items():
                            communities_dict.setdefault(comm_id, set()).add(node)
                        communities = list(communities_dict.values())
                        num_communities = len(communities_dict)
                        
                        # 计算模块化分数
                        modularity_score = community.modularity(partition, undirected_G, weight='weight') if undirected_G.edges() else 0
                        
                        # 统计社区内外权重
                        for u, v, data in G.edges(data=True):
                            comm_u = partition.get(u, -1)
                            comm_v = partition.get(v, -1)
                            if comm_u == comm_v and comm_u != -1:
                                intra_weight += data.get('weight', 1)
                            else:
                                inter_weight += data.get('weight', 1)
                    except Exception as e:
                        print(f"Community detection error: {e}")
                        modularity_score = 0
                else:
                    modularity_score = 0

                # 其他指标计算...
                q1 = G.number_of_nodes()
                q2 = G.number_of_edges()
                q3 = nx.pagerank(G, alpha=0.85)
                q6 = nx.degree_assortativity_coefficient(G)
                q4 = modularity_score

                # 将结果加入缓存
                index_buffer.append([
                    str(w), str(q1), str(q2), str(q3), str(q6), str(q4),
                    str(intra_weight), str(inter_weight), str(num_communities)
                ])

                # Write to CSV when buffer reaches a certain size (e.g., every 100 days)
                if len(index_buffer) >= 100:
                    with open('index.csv', 'a', newline='', encoding='utf-8') as f:
                        writer = csv.writer(f)
                        if f.tell() == 0:  # Check if file is empty to write headers
                            writer.writerow(['Time', 'Num_nodes', 'Num_edges', 'Pagerank','Dessity', 'Modularity',
                                             'intra_weight','str(inter_weight)','str(num_communities)'])
                        writer.writerows(index_buffer)
                    index_buffer.clear()  # Clear the buffer after writing

        # Write any remaining data to the CSV file
        if index_buffer:
            with open('index.csv', 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                if f.tell() == 0:  # Check if file is empty to write headers
                    writer.writerow(['Time', 'Num_nodes', 'Num_edges', 'Pagerank','Dessity', 'Modularity',
                                     'intra_weight','str(inter_weight)','str(num_communities)'])
                writer.writerows(index_buffer)
    
                # Save adjacency matrix for the current week
                timestamp_str = w.strftime("%Y-%m-%d")
                adjacency_matrix = nx.adjacency_matrix(G).todense()
                #pd.DataFrame(adjacency_matrix, index=G.nodes, columns=G.nodes).to_csv(f'G-{timestamp_str}.csv')

    # Save final data to a text file
    with open("normaldata2.txt", "w+", encoding='utf-8') as f:
        f.write(str(makelist))
    # Save the final adjacency matrix
    d = nx.adjacency_matrix(G).todense()
    pd.DataFrame(d, index=G.nodes, columns=G.nodes).to_csv('G.csv')

    return makelist



if __name__ == '__main__':
    print(datetime.datetime.now())
    main2()

2025-02-03 15:56:08.931429


/home/root_gdut/miniconda3/envs/network/lib/python3.9/site-packages/pandas/core/indexes/base.py:7631: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(index_like, name=name, copy=copy)
/home/root_gdut/miniconda3/envs/network/lib/python3.9/site-packages/networkx/algorithms/assortativity/mixing.py:212: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()
/home/root_gdut/miniconda3/envs/network/lib/python3.9/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return (xy * (M - ab)).sum() / np.sqrt(vara * varb)


OSError: [Errno 28] No space left on device

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import datetime
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import re
import numpy as np
from sympy.abc import x
from networkx.algorithms.community import greedy_modularity_communities, modularity
from tqdm import tqdm
from collections import deque

import csv 
def is_valid_id(tweet_id):
    # 检查推文ID是否是有效的（例如不为空，且是数字，或者是合理的ID范围）
    if pd.isna(tweet_id) or tweet_id == "" or not isinstance(tweet_id, (int, float)):
        return False
    return True
def main2():
    # 创建有向图
    G = nx.DiGraph()

    # 读取数据并预处理
    table = pd.read_csv("final_without_quotes.txt", encoding='utf-8')
    table = table.iloc[:, 1:]  # 移除第一列（索引列）

    # 处理时间字段，删除无效或格式错误的记录
    to_drop = []  # 用于存储需要删除的行索引
    for i in range(len(table)):
        try:
            table.loc[i, 'raw_value.created_at'] = pd.to_datetime(table.loc[i, 'raw_value.created_at'])
        except:
            to_drop.append(i)
        if isinstance(table.loc[i, 'raw_value.created_at'], datetime.datetime):
            pass
        else:
            to_drop.append(i)

        if table.loc[i, 'raw_value.created_at'].tzinfo is not None:
            to_drop.append(i)

    # 删除无效的行
    table.drop(to_drop, inplace=True)
    
    table['raw_value.created_at'].sort_values(ascending=True)
    table.to_csv("final_without_quotes.txt", encoding='utf-8')
    table.index = table['raw_value.created_at']

    # 定义数据存储结构
    users = {i: {} for i in set(table['raw_value.user_id_str'].values.tolist())}
    makelist = {}
    date_diff = 30  # 时间差阈值（30天）
    output_dir = "degree"
    edge_timestamps = deque()  # 双端队列缓存边的时间戳和对应的边
    edge_lifetime = 30  # 边的生命周期 (天)
    
    # 用于批量存储index.csv中的数据
    index_buffer = []  # 用于存储一段时间内的写入数据

    # 按天处理数据
    for w, w_t in table.resample('D'):  # 按天分组
        if len(w_t) > 0:  # 如果本天有数据
            for index, row in w_t.iterrows():
                tweet_id = row['raw_value.id_str']  # 获取推文ID
                # 判断推文是否存在交互关系
                if row['raw_value.in_reply_to_status_id_str'] or row['raw_value.quoted_status_id_str']:
                    this_data_datetime = row['raw_value.created_at']
                    
                    # 检查用户ID是否为空
                    user_id = row['raw_value.user_id_str']
                    if pd.isna(user_id) or user_id == "":
                        continue  # 跳过没有有效用户ID的记录

                    # 添加节点到图中
                    G.add_node(tweet_id)  # 添加节点到图中

                    # 遍历缓存中的推文，寻找交互关系
                    for make_i in makelist.keys():
                        diff = (this_data_datetime - makelist[make_i]['created_at']).days
                        if diff < date_diff:  # 如果时间差在范围内
                            # 回复关系
                            if is_valid_id(row['raw_value.in_reply_to_status_id_str']):
                                edge_timestamps.append((makelist[make_i]['id'], row['raw_value.in_reply_to_status_id_str'], this_data_datetime))
                                G.add_edge(makelist[make_i]['id'], row['raw_value.in_reply_to_status_id_str'], weight=1, timestamp=this_data_datetime)

                            if is_valid_id(row['raw_value.quoted_status_id_str']):
                                edge_timestamps.append((makelist[make_i]['id'], row['raw_value.quoted_status_id_str'], this_data_datetime))
                                G.add_edge(makelist[make_i]['id'], row['raw_value.quoted_status_id_str'], weight=1, timestamp=this_data_datetime)

            if row['raw_value.user_id_str'] not in makelist:
                makelist[row['raw_value.user_id_str']] = {
                    'userid': row['raw_value.user_id_str'],
                    'id': tweet_id,
                    'text': row['raw_value.full_text'],
                    'reply': row['raw_value.in_reply_to_status_id_str'],
                    'quoted': row['raw_value.quoted_status_id_str'],
                    'cleantext': row['raw_value.full_text'],
                    'created_at': row['raw_value.created_at']
                }

            # 删除超过30天的边和节点
            to_remove_edges = []
            to_remove_nodes = set()  # 用于存储需要删除的节点

            # 从双端队列中删除过期的边
            while edge_timestamps and (w - edge_timestamps[0][2]).days > edge_lifetime:
                edge_start, edge_end, timestamp = edge_timestamps.popleft()
                if G.has_edge(edge_start, edge_end):
                    G.remove_edge(edge_start, edge_end)
                    to_remove_nodes.add(edge_start)
                    to_remove_nodes.add(edge_end)
                    print(f"已删除边：{(edge_start, edge_end)}  (时间差超过 {date_diff} 天)")

            # 删除节点
            for node in tqdm(to_remove_nodes, desc="删除过期节点", unit="节点"):
                if G.has_node(node):
                    G.remove_node(node)
                    print(f"已删除节点：{node}  (时间差超过 {date_diff} 天)")

            # Save edge information to a text file
            with open("edge_info.txt", "w+", encoding='utf-8') as f:
                f.write("Source,Target,Timestamp\n")  # 写入表头
                for edge in edge_timestamps:
                    source, target, timestamp = edge
                    f.write(f"{source},{target},{timestamp}\n")
    return edge_lifetime


if __name__ == '__main__':
    print(datetime.datetime.now())
    main2()



2025-02-03 17:17:00.633355


/home/root_gdut/miniconda3/envs/network/lib/python3.9/site-packages/pandas/core/indexes/base.py:7631: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(index_like, name=name, copy=copy)
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节

已删除边：(1414494418569158912, 1.4234582977099407e+18)  (时间差超过 30 天)
已删除边：(1418840135500697600, 1.4234582977099407e+18)  (时间差超过 30 天)
已删除边：(1414494418569158912, 1.4234583792346276e+18)  (时间差超过 30 天)
已删除边：(1418840135500697600, 1.4234583792346276e+18)  (时间差超过 30 天)
已删除边：(1414494418569158912, 1.4234584834085396e+18)  (时间差超过 30 天)
已删除边：(1418840135500697600, 1.4234584834085396e+18)  (时间差超过 30 天)


删除过期节点: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 21799.92节点/s]


已删除节点：1414494418569158912  (时间差超过 30 天)
已删除节点：1.4234582977099407e+18  (时间差超过 30 天)
已删除节点：1418840135500697600  (时间差超过 30 天)
已删除节点：1.4234584834085396e+18  (时间差超过 30 天)
已删除节点：1.4234583792346276e+18  (时间差超过 30 天)


...

已删除边：(1533230623048888320, 1.533247920610644e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5333373651622748e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5334487497791406e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.533403339609854e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5334505936203448e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.533579869828698e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.533645930532307e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5336925695821906e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.533717646084313e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5337481136717865e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5337485048619786e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5337497950339973e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5337621990506043e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5336239190765814e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5337922649834824e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5

已删除边：(1537938401273446400, 1.5394728538498703e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5394946666214195e+18)  (时间差超过 30 天)
已删除边：(1536019389857746944, 1.5394946666214195e+18)  (时间差超过 30 天)
已删除边：(1536854772858335232, 1.5394946666214195e+18)  (时间差超过 30 天)
已删除边：(1537145637803675648, 1.5394946666214195e+18)  (时间差超过 30 天)
已删除边：(1537938401273446400, 1.5394946666214195e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5395053797337375e+18)  (时间差超过 30 天)
已删除边：(1536019389857746944, 1.5395053797337375e+18)  (时间差超过 30 天)
已删除边：(1536854772858335232, 1.5395053797337375e+18)  (时间差超过 30 天)
已删除边：(1537145637803675648, 1.5395053797337375e+18)  (时间差超过 30 天)
已删除边：(1537938401273446400, 1.5395053797337375e+18)  (时间差超过 30 天)
已删除边：(1533230623048888320, 1.5395361128293376e+18)  (时间差超过 30 天)
已删除边：(1536019389857746944, 1.5395361128293376e+18)  (时间差超过 30 天)
已删除边：(1536854772858335232, 1.5395361128293376e+18)  (时间差超过 30 天)
已删除边：(1537145637803675648, 1.5395361128293376e+18)  (时间差超过 30 天)
已删除边：(1537938401273446400

删除过期节点: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 212/212 [00:00<00:00, 120307.46节点/s]


已删除节点：1533230623048888320  (时间差超过 30 天)
已删除节点：1.533247920610644e+18  (时间差超过 30 天)
已删除节点：1.5333373651622748e+18  (时间差超过 30 天)
已删除节点：1.533403339609854e+18  (时间差超过 30 天)
已删除节点：1.5334487497791406e+18  (时间差超过 30 天)
已删除节点：1.5334505936203448e+18  (时间差超过 30 天)
已删除节点：1.533579869828698e+18  (时间差超过 30 天)
已删除节点：1.533645930532307e+18  (时间差超过 30 天)
已删除节点：1.5336925695821906e+18  (时间差超过 30 天)
已删除节点：1.533717646084313e+18  (时间差超过 30 天)
已删除节点：1.5341611143508787e+18  (时间差超过 30 天)
已删除节点：1.5360119319154117e+18  (时间差超过 30 天)
已删除节点：1.5372756888894917e+18  (时间差超过 30 天)
已删除节点：1.542682485137351e+18  (时间差超过 30 天)
已删除节点：1.539785315006554e+18  (时间差超过 30 天)
已删除节点：1.5408602928541942e+18  (时间差超过 30 天)
已删除节点：1.541612639385559e+18  (时间差超过 30 天)
已删除节点：1.542025106288812e+18  (时间差超过 30 天)
已删除节点：1.5368705395451617e+18  (时间差超过 30 天)
已删除节点：1.53705215086993e+18  (时间差超过 30 天)
已删除节点：1.5381694325599887e+18  (时间差超过 30 天)
已删除节点：1.5388806081178624e+18  (时间差超过 30 天)
已删除节点：1.5385392718666916e+18  (时间差超过 30 天)
已删除节点：1.5405741415309107e

删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]
删除过期节点: 0节点 [00:00, ?节点/s]


In [2]:
print('ok')

ok
